# SIMULATION FOR THESIS 

In [21]:
## Configurable Simulation Parameters
NUM_RUNS = 2500             ## Set the number of simulation runs
SPEED_MULTIPLIER = 2        ## How fast the simulation runs (higher = faster)
DISPLAY_GRAPHICS = False    ## Set to False for fastest processing with no visuals
DATA_DIRECTORY = r"C:\Users\cappeale\Thesis_Game"  ## Directory to save simulation data
AUTONOMOUS_MODE = True   ## If True, runs the simulation autonomously without printing the interface

In [22]:
import pygame
import random
import math
import csv
import os
import time
import datetime
import tempfile
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum, auto
from IPython.display import display, clear_output
from ipywidgets import Button, HBox, VBox, Output, IntSlider, FloatSlider, Checkbox, HTML

## Force matplotlib to use inline mode
plt.rcParams['figure.figsize'] = (10, 7)

## Special initialization for pygame in Jupyter
import os
os.environ['SDL_VIDEODRIVER'] = 'dummy'  # Headless pygame
pygame.init()

(5, 0)

In [23]:
## Define constants and enums

## Visual and Game constraings
WIDTH, HEIGHT = 800, 600
ROOM_WIDTH, ROOM_HEIGHT = 300, 300
PLAYER_SIZE = 20
NPC_SIZE = 15
ENEMY_SIZE = 20
RESOURCE_SIZE = 15
FPS = 60
TOTAL_FRAMES = FPS * 180  ## 3 minutes of gameplay

## Color patterns
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
PURPLE = (128, 0, 128)
BROWN = (165, 42, 42)
GRAY = (128, 128, 128)

## Game parameters
MAX_HEALTH = 100
ENEMY_DAMAGE = 30  
RESOURCE_HEAL = 5
NOTIFY_RADIUS = 100
ENEMY_DETECTION_RADIUS = 180
ATTACK_RADIUS = 40

## Enums for categoricals
class PlayerAction(Enum):
    IDLE = "idle"
    MOVE = "move"
    ATTACK = "attack"

class NPCEmotion(Enum):
    ANTICIPATION = "anticipation"
    HAPPINESS = "happiness"
    FEAR = "fear"
    ANGER = "anger"
    SURPRISE = "surprise"
    SADNESS = "sadness"

class NPCReaction(Enum):
    FOLLOW = "follow"
    NOTIFY_RESOURCE = "notify_resource"
    NOTIFY_DANGER = "notify_danger"
    ATTACK_ENEMY = "attack_enemy"
    NOTIFY_SURPRISE = "notify_surprise"
    PROVIDE_HEALING = "provide_healing"

## Function for generating the CSV filename
def get_unique_filename(base_path):
    return os.path.join(base_path, "simulation_data_all_runs.csv")


In [24]:
# Game class defintion
class Game:
    def __init__(self, run_id=1, max_runs=NUM_RUNS, speed_multiplier=SPEED_MULTIPLIER):
        ## Create a surface for rendering without direct display
        self.screen = pygame.Surface((WIDTH, HEIGHT))
        self.clock = pygame.time.Clock()
        self.font = pygame.font.SysFont(None, 24)
        
        self.run_id = run_id
        self.max_runs = max_runs
        self.frame = 0
        self.paused = False
        self.speed_multiplier = speed_multiplier
        self.running = True
        self.game_over = False
        
        ## Try to create the data directory
        try:
            os.makedirs(DATA_DIRECTORY, exist_ok=True)
            self.output_dir = DATA_DIRECTORY
        except:
            self.output_dir = tempfile.gettempdir()
            print(f"Could not access {DATA_DIRECTORY}, using {self.output_dir} instead")
        
        self.reset()
        
        ## CSV data collection with a single CSV file for all runs
        self.csv_filename = get_unique_filename(self.output_dir)

        ## Added four new columns: lagged_player_action, resources_collected, enemies_killed, level
        self.csv_header = [
            "Time", "Player_x", "Player_y", "NPC_x", "NPC_y",
            "Player_health", "Enemy_proximity", "Resource_proximity",
            "Player_action", "lagged_player_action", 
            "resources_collected", "enemies_killed", "level",
            "NPC_emotion_lagged", "NPC_emotion",
            "NPC_reaction"
        ]
        
        if not os.path.exists(self.csv_filename):
            try:
                with open(self.csv_filename, 'w', newline='') as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerow(self.csv_header)
            except PermissionError:
                self.output_dir = tempfile.gettempdir()
                print(f"Permission error writing to original location. Using temp directory instead: {self.output_dir}")
                self.csv_filename = get_unique_filename(self.output_dir)
                with open(self.csv_filename, 'w', newline='') as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerow(self.csv_header)
    
    def reset(self):
        self.running = True
        self.game_over = False
        self.resources_collected = 0
        self.enemies_killed = 0
        self.door_opened = False
        self.frame = 0
        self.current_room = 1  ## Start in room 1
        
        ## Track player's previous action (lagged) -->  Initialize with IDLE each reset
        self.lagged_player_action = PlayerAction.IDLE
        
        self.player = {
            "x": 150,
            "y": 150,
            "health": 100,
            "action": PlayerAction.IDLE,
            "speed": 5,
            "resources_collected": 0,
            "enemies_killed": 0
        }
        
        self.npc = {
            "x": 100,
            "y": 100,
            "emotion": NPCEmotion.ANTICIPATION,
            "emotion_lagged": NPCEmotion.ANTICIPATION,
            "reaction": NPCReaction.FOLLOW,
            "speed": 3
        }
        
        self.enemies = []
        for _ in range(5):
            self.enemies.append({
                "x": random.randint(50 + 20, 50 + ROOM_WIDTH - 20),
                "y": random.randint(50 + 20, 50 + ROOM_HEIGHT - 20),
                "speed": random.uniform(1, 2),
                "alive": True,
                "room": 1
            })
        
        for _ in range(5):
            self.enemies.append({
                "x": random.randint(450 + 20, 450 + ROOM_WIDTH - 20),
                "y": random.randint(50 + 20, 50 + ROOM_HEIGHT - 20),
                "speed": random.uniform(1, 2),
                "alive": True,
                "room": 2
            })
        
        self.resources = []
        for _ in range(2):
            self.resources.append({
                "x": random.randint(50 + 20, 50 + ROOM_WIDTH - 20),
                "y": random.randint(50 + 20, 50 + ROOM_HEIGHT - 20),
                "collected": False,
                "room": 1
            })
        
        for _ in range(2):
            self.resources.append({
                "x": random.randint(450 + 20, 450 + ROOM_WIDTH - 20),
                "y": random.randint(50 + 20, 50 + ROOM_HEIGHT - 20),
                "collected": False,
                "room": 2
            })
        
        self.rooms = {
            1: {"x": 50, "y": 50, "width": ROOM_WIDTH, "height": ROOM_HEIGHT},
            2: {"x": 450, "y": 50, "width": ROOM_WIDTH, "height": ROOM_HEIGHT}
        }
        
        self.door = {
            "x": 340,
            "y": 50 + ROOM_HEIGHT // 2 - 25,
            "width": 100,
            "height": 50,
            "open": False
        }
        
        self.exit = {
            "x": 700,
            "y": 200,
            "width": 50,
            "height": 50,
            "room": 2
        }
        
        room2_resources = sum(1 for resource in self.resources if resource["room"] == 2 and not resource["collected"])
        room2_enemies = sum(1 for enemy in self.enemies if enemy["room"] == 2 and enemy["alive"])
        
        print(f"Room 2 setup: {room2_resources} resources and {room2_enemies} enemies")
    
    def get_nearest_enemy_distance(self):
        min_distance = float('inf')
        for enemy in self.enemies:
            if enemy["room"] == self.current_room and enemy["alive"]:
                distance = math.sqrt((self.player["x"] - enemy["x"])**2 + (self.player["y"] - enemy["y"])**2)
                min_distance = min(min_distance, distance)
        return min_distance if min_distance != float('inf') else 1000
    
    def get_nearest_resource_distance(self):
        min_distance = float('inf')
        for resource in self.resources:
            if resource["room"] == self.current_room and not resource["collected"]:
                distance = math.sqrt((self.player["x"] - resource["x"])**2 + (self.player["y"] - resource["y"])**2)
                min_distance = min(min_distance, distance)
        return min_distance if min_distance != float('inf') else 1000
    
    def update_npc_emotion(self):
        self.npc["emotion_lagged"] = self.npc["emotion"]
        self.npc["emotion"] = NPCEmotion.ANTICIPATION
        self.npc["reaction"] = NPCReaction.FOLLOW
        
        enemy_distance = self.get_nearest_enemy_distance()
        resource_distance = self.get_nearest_resource_distance()
        
        if resource_distance < NOTIFY_RADIUS:
            self.npc["emotion"] = NPCEmotion.HAPPINESS
            self.npc["reaction"] = NPCReaction.NOTIFY_RESOURCE
        
        if enemy_distance < (ENEMY_DETECTION_RADIUS * 0.3):
            self.npc["emotion"] = NPCEmotion.FEAR
            self.npc["reaction"] = NPCReaction.NOTIFY_DANGER
        
        for enemy in self.enemies:
            if (enemy["room"] == self.current_room and enemy["alive"] and 
                math.sqrt((self.player["x"] - enemy["x"])**2 + (self.player["y"] - enemy["y"])**2) < ATTACK_RADIUS):
                self.npc["emotion"] = NPCEmotion.ANGER
                self.npc["reaction"] = NPCReaction.ATTACK_ENEMY
                break
        
        if (self.player["action"] == PlayerAction.ATTACK and 
            math.sqrt((self.player["x"] - self.npc["x"])**2 + (self.player["y"] - self.npc["y"])**2) < ATTACK_RADIUS):
            self.npc["emotion"] = NPCEmotion.SURPRISE
            self.npc["reaction"] = NPCReaction.NOTIFY_SURPRISE
        
        if self.player["health"] <= 30:
            self.npc["emotion"] = NPCEmotion.SADNESS
            self.npc["reaction"] = NPCReaction.PROVIDE_HEALING
    
    def update_npc_position(self):
        IDEAL_DISTANCE = 50
        MIN_DISTANCE = 30
        
        if self.npc["reaction"] == NPCReaction.FOLLOW:
            dx = self.player["x"] - self.npc["x"]
            dy = self.player["y"] - self.npc["y"]
            distance = math.sqrt(dx**2 + dy**2)
            
            if distance > 0:
                dx /= distance
                dy /= distance
                
                if distance > IDEAL_DISTANCE + 20:
                    speed_factor = 1.5
                elif distance < MIN_DISTANCE:
                    speed_factor = -0.5
                elif distance > IDEAL_DISTANCE:
                    speed_factor = 1.0
                else:
                    speed_factor = 0.5
                
                self.npc["x"] += dx * self.npc["speed"] * speed_factor
                self.npc["y"] += dy * self.npc["speed"] * speed_factor
        
        elif self.npc["reaction"] == NPCReaction.NOTIFY_RESOURCE or self.npc["reaction"] == NPCReaction.NOTIFY_DANGER:
            dx = self.player["x"] - self.npc["x"]
            dy = self.player["y"] - self.npc["y"]
            distance = math.sqrt(dx**2 + dy**2)
            
            if distance > IDEAL_DISTANCE * 1.5:
                dx /= distance
                dy /= distance
                self.npc["x"] += dx * self.npc["speed"] * 1.2
                self.npc["y"] += dy * self.npc["speed"] * 1.2
        
        elif self.npc["reaction"] == NPCReaction.ATTACK_ENEMY:
            nearest_enemy = None
            min_distance = float('inf')
            
            for i, enemy in enumerate(self.enemies):
                if enemy["room"] == self.current_room and enemy["alive"]:
                    distance = math.sqrt((self.npc["x"] - enemy["x"])**2 + (self.npc["y"] - enemy["y"])**2)
                    if distance < min_distance:
                        min_distance = distance
                        nearest_enemy = i
            
            if nearest_enemy is not None:
                if min_distance > ATTACK_RADIUS:
                    dx = self.enemies[nearest_enemy]["x"] - self.npc["x"]
                    dy = self.enemies[nearest_enemy]["y"] - self.npc["y"]
                    dist = math.sqrt(dx**2 + dy**2)
                    if dist > 0:
                        dx /= dist
                        dy /= dist
                        self.npc["x"] += dx * self.npc["speed"] * 1.2
                        self.npc["y"] += dy * self.npc["speed"] * 1.2
                elif min_distance < ATTACK_RADIUS:
                    self.enemies[nearest_enemy]["alive"] = False
                    self.player["enemies_killed"] += 1
        
        elif self.npc["reaction"] == NPCReaction.PROVIDE_HEALING:
            dx = self.player["x"] - self.npc["x"]
            dy = self.player["y"] - self.npc["y"]
            distance = math.sqrt(dx**2 + dy**2)
            
            if distance > ATTACK_RADIUS:
                dx /= distance
                dy /= distance
                self.npc["x"] += dx * self.npc["speed"] * 1.5
                self.npc["y"] += dy * self.npc["speed"] * 1.5
            else:
                self.player["health"] = min(self.player["health"] + 10, 100)
        
        current_room = self.rooms[self.current_room]
        self.npc["x"] = max(current_room["x"] + NPC_SIZE//2, min(self.npc["x"], current_room["x"] + current_room["width"] - NPC_SIZE//2))
        self.npc["y"] = max(current_room["y"] + NPC_SIZE//2, min(self.npc["y"], current_room["y"] + current_room["height"] - NPC_SIZE//2))
    
    def update_enemies(self):
        for enemy in self.enemies:
            if enemy["room"] == self.current_room and enemy["alive"]:
                enemy["x"] += random.uniform(-enemy["speed"], enemy["speed"])
                enemy["y"] += random.uniform(-enemy["speed"], enemy["speed"])
                
                enemy["x"] = max(self.rooms[self.current_room]["x"], min(enemy["x"], self.rooms[self.current_room]["x"] + self.rooms[self.current_room]["width"]))
                enemy["y"] = max(self.rooms[self.current_room]["y"], min(enemy["y"], self.rooms[self.current_room]["y"] + self.rooms[self.current_room]["height"]))
                
                player_distance = math.sqrt((self.player["x"] - enemy["x"])**2 + (self.player["y"] - enemy["y"])**2)
                if player_distance < ENEMY_DETECTION_RADIUS:
                    dx = self.player["x"] - enemy["x"]
                    dy = self.player["y"] - enemy["y"]
                    if player_distance > 0:
                        dx /= player_distance
                        dy /= player_distance
                        enemy["x"] += dx * enemy["speed"]
                        enemy["y"] += dy * enemy["speed"]
                
                if player_distance < ATTACK_RADIUS:
                    self.player["health"] -= ENEMY_DAMAGE / (FPS/4)
                    self.player["health"] = max(0, self.player["health"])
    
    def check_resource_collection(self):
        for resource in self.resources:
            if (resource["room"] == self.current_room and not resource["collected"] and
                math.sqrt((self.player["x"] - resource["x"])**2 + (self.player["y"] - resource["y"])**2) < ATTACK_RADIUS):
                resource["collected"] = True
                self.player["resources_collected"] += 1
                self.player["health"] = min(self.player["health"] + (RESOURCE_HEAL * 0.5), 100)
                
                if self.current_room == 1 and self.player["resources_collected"] >= 2:
                    self.door["open"] = True
                    print("Door opened! Player can now move to Room 2")
    
    def check_door_interaction(self):
        player_rect = pygame.Rect(self.player["x"] - PLAYER_SIZE/2, self.player["y"] - PLAYER_SIZE/2, PLAYER_SIZE, PLAYER_SIZE)
        door_rect = pygame.Rect(self.door["x"], self.door["y"], self.door["width"], self.door["height"])
        
        if player_rect.colliderect(door_rect):
            if self.door["open"]:
                if self.current_room == 1:
                    print("Transitioning to Room 2")
                    self.current_room = 2
                    self.player["x"] = self.rooms[2]["x"] + 50
                    self.player["y"] = self.rooms[2]["y"] + ROOM_HEIGHT/2
                    self.npc["x"] = self.player["x"] - 30
                    self.npc["y"] = self.player["y"]
                    self.player["speed"] = 5
                else:
                    print("Transitioning to Room 1")
                    self.current_room = 1
                    self.player["x"] = self.rooms[1]["x"] + ROOM_WIDTH - 50
                    self.player["y"] = self.rooms[1]["y"] + ROOM_HEIGHT/2
                    self.npc["x"] = self.player["x"] - 30
                    self.npc["y"] = self.player["y"]
                    self.player["speed"] = 5
            else:
                if self.player["action"] == PlayerAction.MOVE:
                    if self.player["x"] < self.door["x"]:
                        self.player["x"] -= 5
                    else:
                        self.player["x"] += 5
    
    def check_exit_interaction(self):
        if self.current_room == 2:
            player_rect = pygame.Rect(self.player["x"] - PLAYER_SIZE/2, self.player["y"] - PLAYER_SIZE/2, PLAYER_SIZE, PLAYER_SIZE)
            exit_rect = pygame.Rect(self.exit["x"], self.exit["y"], self.exit["width"], self.exit["height"])
            
            if player_rect.colliderect(exit_rect):
                if self.player["enemies_killed"] >= 5 and self.player["resources_collected"] >= 4:
                    print(f"Run {self.run_id} completed successfully!")
                    self.game_over = True
                else:
                    print(f"Exit reached but objectives not complete: {self.player['resources_collected']}/4 resources, {self.player['enemies_killed']}/5 enemies")
    
    def check_attack(self):
        if self.player["action"] == PlayerAction.ATTACK:
            for i, enemy in enumerate(self.enemies):
                if enemy["room"] == self.current_room and enemy["alive"]:
                    if math.sqrt((self.player["x"] - enemy["x"])**2 + (self.player["y"] - enemy["y"])**2) < ATTACK_RADIUS:
                        enemy["alive"] = False
                        self.player["enemies_killed"] += 1
                        print(f"Enemy killed! Total: {self.player['enemies_killed']}/5")
    
    def update_player_movement(self):
        """
        Hard-coded player behavior that follows optimal objectives:
            1. Collect resources to open door (if in room 1)
            2. Avoid enemies when health is low
            3. Attack enemies when health is sufficient
            4. Move to exit when all objectives are met
        """
        self.player["action"] = PlayerAction.IDLE
        current_room = self.rooms[self.current_room]
        room_center_x = current_room["x"] + current_room["width"] / 2
        room_center_y = current_room["y"] + current_room["height"] / 2
        target_x, target_y = room_center_x, room_center_y
        target_found = False
        target_type = "explore"
        
        # Add random chance for behaviors that trigger surprise and sadness
        random_behavior = random.random()
        
        # Occasionally create opportunities for SURPRISE emotion (~30% chance)
        if random_behavior < 0.30:
            # Calculate distance between player and NPC
            npc_distance = math.sqrt((self.player["x"] - self.npc["x"])**2 + (self.player["y"] - self.npc["y"])**2)
            
            # If close to NPC, occasionally attack (creating surprise)
            if npc_distance < ATTACK_RADIUS * 2.5:
                # Move toward NPC and prepare to attack
                target_x, target_y = self.npc["x"], self.npc["y"]
                target_type = "npc_vicinity"
                target_found = True
        
        # Occasional risk-taking behavior (less health avoidance) to enable SADNESS (~30% chance)
        take_health_risk = random_behavior > 0.10 and random_behavior < 0.30 and self.player["health"] > 10 and self.player["health"] < 40
        
        if not target_found:  # Only proceed with normal logic if we didn't set special behavior
            if self.current_room == 1:
                if self.door["open"] and self.player["enemies_killed"] >= 2:
                    target_x = self.door["x"] + self.door["width"]/2
                    target_y = self.door["y"] + self.door["height"]/2
                    target_type = "door"
                    target_found = True
                    self.player["speed"] = 7
                elif self.player["enemies_killed"] < 2:
                    closest_enemy = None
                    min_distance = float('inf')
                    for enemy in self.enemies:
                        if enemy["room"] == 1 and enemy["alive"]:
                            distance = math.sqrt((self.player["x"] - enemy["x"])**2 + (self.player["y"] - enemy["y"])**2)
                            if distance < min_distance:
                                min_distance = distance
                                closest_enemy = enemy
                    if closest_enemy is not None:
                        target_x, target_y = closest_enemy["x"], closest_enemy["y"]
                        target_type = "enemy"
                        target_found = True
                if not target_found:
                    for resource in self.resources:
                        if resource["room"] == 1 and not resource["collected"]:
                            target_x, target_y = resource["x"], resource["y"]
                            target_found = True
                            target_type = "resource"
                            break
                    if not target_found and self.player["resources_collected"] >= 2:
                        target_x, target_y = self.door["x"] + self.door["width"]/2, self.door["y"] + self.door["height"]/2
                        target_type = "door"
            elif self.current_room == 2:
                if self.player["resources_collected"] < 4:
                    for resource in self.resources:
                        if resource["room"] == 2 and not resource["collected"]:
                            target_x, target_y = resource["x"], resource["y"]
                            target_found = True
                            target_type = "resource"
                            break
                if not target_found and self.player["enemies_killed"] < 5:
                    closest_enemy = None
                    min_distance = float('inf')
                    for enemy in self.enemies:
                        if enemy["room"] == 2 and enemy["alive"]:
                            distance = math.sqrt((self.player["x"] - enemy["x"])**2 + (self.player["y"] - enemy["y"])**2)
                            if distance < min_distance:
                                min_distance = distance
                                closest_enemy = enemy
                    if closest_enemy:
                        target_x, target_y = closest_enemy["x"], closest_enemy["y"]
                        target_type = "enemy"
                if not target_found and self.player["resources_collected"] >= 4 and self.player["enemies_killed"] >= 5:
                    target_x, target_y = self.exit["x"] + self.exit["width"]/2, self.exit["y"] + self.exit["height"]/2
                    target_type = "exit"
        
        extremely_close_threshold = ATTACK_RADIUS / 2
        for enemy in self.enemies:
            if enemy["room"] == self.current_room and enemy["alive"]:
                distance = math.sqrt((self.player["x"] - enemy["x"])**2 + (self.player["y"] - enemy["y"])**2)
                if distance < extremely_close_threshold:
                    target_x, target_y = enemy["x"], enemy["y"]
                    target_type = "enemy"
                    break
        
        ## Fundamental aspect that was tuned quite often: the health avoidance behavior to occasionally take risks and showcase surprise / fear more
        if self.player["health"] < 30 and not take_health_risk:
            for enemy in self.enemies:
                if enemy["room"] == self.current_room and enemy["alive"]:
                    distance = math.sqrt((self.player["x"] - enemy["x"])**2 + (self.player["y"] - enemy["y"])**2)
                    if distance < ENEMY_DETECTION_RADIUS:
                        flee_x = self.player["x"] + (self.player["x"] - enemy["x"])
                        flee_y = self.player["y"] + (self.player["y"] - enemy["y"])
                        target_x = 0.3 * flee_x + 0.7 * target_x
                        target_y = 0.3 * flee_y + 0.7 * target_y
        
        dx = target_x - self.player["x"]
        dy = target_y - self.player["y"]
        distance = math.sqrt(dx**2 + dy**2)
        
        if distance > 5:
            dx /= distance
            dy /= distance
            self.player["x"] += dx * self.player["speed"]
            self.player["y"] += dy * self.player["speed"]
            self.player["action"] = PlayerAction.MOVE
            if target_type == "enemy" and distance < ATTACK_RADIUS * 1.2 and self.player["health"] > 30:
                self.player["action"] = PlayerAction.ATTACK
                self.check_attack()
        else:
            if target_type == "enemy":
                self.player["action"] = PlayerAction.ATTACK
                self.check_attack()
        
        ## Here too surprise was tuned more often to ensure sufficiently realistic surprise measures
        if target_type == "npc_vicinity" and random.random() < 0.75: 
            distance_to_npc = math.sqrt((self.player["x"] - self.npc["x"])**2 + (self.player["y"] - self.npc["y"])**2)
            if distance_to_npc < ATTACK_RADIUS * 1.5:
                self.player["action"] = PlayerAction.ATTACK
        
        self.player["x"] = max(current_room["x"] + PLAYER_SIZE//2, min(self.player["x"], current_room["x"] + current_room["width"] - PLAYER_SIZE//2))
        self.player["y"] = max(current_room["y"] + PLAYER_SIZE//2, min(self.player["y"], current_room["y"] + current_room["height"] - PLAYER_SIZE//2))
    
    def record_data(self, old_action):
        try:
            with open(self.csv_filename, 'a', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow([
                    self.frame,
                    int(self.player["x"]),
                    int(self.player["y"]),
                    int(self.npc["x"]),
                    int(self.npc["y"]),
                    int(self.player["health"]),
                    float(self.get_nearest_enemy_distance()),
                    float(self.get_nearest_resource_distance()),
                    self.player["action"].value,       ## current action
                    old_action.value,                  ## lagged_player_action
                    self.player["resources_collected"],
                    self.player["enemies_killed"],
                    self.current_room,                 ## level (room number)
                    self.npc["emotion_lagged"].value,
                    self.npc["emotion"].value,
                    self.npc["reaction"].value
                ])
        except Exception as e:
            print(f"Error writing to CSV: {e}")
    
    def draw(self):
        self.screen.fill(BLACK)
        
        for room_id, room in self.rooms.items():
            color = PURPLE if room_id == self.current_room else GRAY
            pygame.draw.rect(self.screen, color, (room["x"], room["y"], room["width"], room["height"]), 2)
        
        if self.current_room == 1 or self.current_room == 2:
            room = self.rooms[self.current_room]
            
            for resource in self.resources:
                if resource["room"] == self.current_room and not resource["collected"]:
                    pygame.draw.rect(self.screen, YELLOW, 
                                    (resource["x"] - RESOURCE_SIZE/2, 
                                     resource["y"] - RESOURCE_SIZE/2, 
                                     RESOURCE_SIZE, RESOURCE_SIZE))
            
            for enemy in self.enemies:
                if enemy["room"] == self.current_room and enemy["alive"]:
                    pygame.draw.circle(self.screen, RED, (int(enemy["x"]), int(enemy["y"])), ENEMY_SIZE//2)
            
            if self.current_room == 2:
                pygame.draw.rect(self.screen, GREEN, 
                                (self.exit["x"], self.exit["y"], 
                                 self.exit["width"], self.exit["height"]))
            
            pygame.draw.circle(self.screen, BLUE, (int(self.player["x"]), int(self.player["y"])), PLAYER_SIZE//2)
            pygame.draw.circle(self.screen, GREEN, (int(self.npc["x"]), int(self.npc["y"])), NPC_SIZE//2)
            
            emotion_symbols = {
                NPCEmotion.ANTICIPATION: "...",
                NPCEmotion.HAPPINESS: ":)",
                NPCEmotion.FEAR: "!!",
                NPCEmotion.ANGER: "!!",
                NPCEmotion.SURPRISE: "?!",
                NPCEmotion.SADNESS: ":("
            }
            
            emotion_color = {
                NPCEmotion.ANTICIPATION: YELLOW,
                NPCEmotion.HAPPINESS: GREEN,
                NPCEmotion.FEAR: WHITE,
                NPCEmotion.ANGER: RED,
                NPCEmotion.SURPRISE: PURPLE,
                NPCEmotion.SADNESS: BLUE
            }
            
            bubble_radius = 15
            pygame.draw.circle(self.screen, WHITE, 
                             (int(self.npc["x"]), int(self.npc["y"] - 25)), 
                             bubble_radius)
            
            emotion_text = self.font.render(emotion_symbols[self.npc["emotion"]], True, emotion_color[self.npc["emotion"]])
            emotion_rect = emotion_text.get_rect(center=(int(self.npc["x"]), int(self.npc["y"] - 25)))
            self.screen.blit(emotion_text, emotion_rect)
            
            if self.npc["reaction"] == NPCReaction.NOTIFY_RESOURCE:
                nearest_resource = None
                min_distance = float('inf')
                for resource in self.resources:
                    if resource["room"] == self.current_room and not resource["collected"]:
                        distance = math.sqrt((self.npc["x"] - resource["x"])**2 + (self.npc["y"] - resource["y"])**2)
                        if distance < min_distance:
                            min_distance = distance
                            nearest_resource = resource
                if nearest_resource:
                    pygame.draw.line(self.screen, YELLOW, 
                                    (int(self.npc["x"]), int(self.npc["y"])),
                                    (int(nearest_resource["x"]), int(nearest_resource["y"])),
                                    2)
            
            elif self.npc["reaction"] == NPCReaction.NOTIFY_DANGER:
                danger_text = self.font.render("DANGER", True, RED)
                self.screen.blit(danger_text, (int(self.npc["x"]) + 15, int(self.npc["y"]) - 10))
            
            elif self.npc["reaction"] == NPCReaction.ATTACK_ENEMY:
                pygame.draw.circle(self.screen, RED, 
                                 (int(self.npc["x"]), int(self.npc["y"])),
                                 NPC_SIZE, 2)
            
            elif self.npc["reaction"] == NPCReaction.PROVIDE_HEALING:
                pygame.draw.circle(self.screen, GREEN, 
                                 (int(self.npc["x"]), int(self.npc["y"])),
                                 NPC_SIZE + 5, 2)
        
        door_color = GREEN if self.door["open"] else RED
        pygame.draw.rect(self.screen, door_color, 
                        (self.door["x"], self.door["y"], 
                         self.door["width"], self.door["height"]))
        
        pygame.draw.rect(self.screen, RED, (20, 20, 100, 20))
        pygame.draw.rect(self.screen, GREEN, (20, 20, self.player["health"], 20))
        health_text = self.font.render(f"Health: {int(self.player['health'])}", True, WHITE)
        self.screen.blit(health_text, (20, 45))
        
        resources_text = self.font.render(f"Resources: {self.player['resources_collected']}/4", True, WHITE)
        self.screen.blit(resources_text, (20, 70))
        
        enemies_text = self.font.render(f"Enemies: {self.player['enemies_killed']}/5", True, WHITE)
        self.screen.blit(enemies_text, (20, 95))
        
        room_text = self.font.render(f"Room: {self.current_room}", True, WHITE)
        self.screen.blit(room_text, (20, 120))
        
        emotion_text = self.font.render(f"NPC: {self.npc['emotion'].value}", True, WHITE)
        self.screen.blit(emotion_text, (20, 145))
        
        reaction_text = self.font.render(f"NPC Reaction: {self.npc['reaction'].value}", True, WHITE)
        self.screen.blit(reaction_text, (20, 170))
        
        frame_text = self.font.render(f"Frame: {self.frame}", True, WHITE)
        self.screen.blit(frame_text, (20, 195))
        
        run_text = self.font.render(f"Run: {self.run_id}/{self.max_runs}", True, WHITE)
        self.screen.blit(run_text, (20, 220))
        
        door_text = self.font.render("Door: " + ("Open" if self.door["open"] else "Closed"), True, WHITE)
        self.screen.blit(door_text, (20, 245))
        
        return pygame.surfarray.array3d(self.screen).swapaxes(0, 1)
    
    def update(self):
        if hasattr(self, 'paused') and self.paused:
            return
        
        frames_to_advance = int(self.speed_multiplier) if hasattr(self, 'speed_multiplier') else 1
        
        for _ in range(frames_to_advance):
            old_action = self.lagged_player_action
            self.frame += 1
            self.update_player_movement()
            self.update_npc_emotion()
            self.update_npc_position()
            self.update_enemies()
            self.check_resource_collection()
            self.check_door_interaction()
            self.check_exit_interaction()
            self.record_data(old_action)
            self.lagged_player_action = self.player["action"]
            if self.player["health"] <= 0:
                print(f"Game over! Player died in run {self.run_id}")
                self.game_over = True
                break
            if self.frame >= TOTAL_FRAMES:
                print(f"Time's up for run {self.run_id}!")
                self.game_over = True
                break
    
    def step(self):
        if not self.game_over and self.running:
            self.update()
            return self.draw(), self.game_over
        else:
            if self.game_over:
                if self.run_id < self.max_runs:
                    self.run_id += 1
                    self.reset()
                    self.game_over = False
                    return self.draw(), False
                else:
                    print("All runs completed!")
                    return self.draw(), True
            return self.draw(), True
    
    def run_batch(self, num_steps=100):
        for _ in range(num_steps):
            if not self.game_over and self.running:
                self.update()
            else:
                if self.game_over:
                    if self.run_id < self.max_runs:
                        self.run_id += 1
                        self.reset()
                        self.game_over = False
                    else:
                        print("All runs completed!")
                        break
        return self.draw(), self.game_over


In [25]:
## Create interactive simulation interface for manual testing of the simulation
class SimulationInterface:
    def __init__(self, run_id=1, max_runs=NUM_RUNS, speed=SPEED_MULTIPLIER, display_graphics=DISPLAY_GRAPHICS):
        self.game = Game(run_id=run_id, max_runs=max_runs, speed_multiplier=speed)
        self.output = Output()
        self.paused = False
        self.running = False
        self.speed = speed
        self.display_graphics = display_graphics
        
        self.start_button = Button(description="Start/Stop", button_style='success')
        self.start_button.on_click(self.toggle_simulation)
        
        self.step_button = Button(description="Step", button_style='info')
        self.step_button.on_click(self.step_simulation)
        
        self.reset_button = Button(description="Reset", button_style='warning')
        self.reset_button.on_click(self.reset_simulation)
        
        self.batch_button = Button(description="Run 100 Steps", button_style='primary')
        self.batch_button.on_click(self.run_batch)
        
        self.speed_slider = FloatSlider(
            value=speed,
            min=0.1,
            max=10.0,
            step=0.1,
            description='Speed:',
            continuous_update=False
        )
        self.speed_slider.observe(self.update_speed, names='value')
        
        self.run_slider = IntSlider(
            value=max_runs,
            min=1,
            max=100,
            step=1,
            description='Max Runs:',
            continuous_update=False
        )
        self.run_slider.observe(self.update_max_runs, names='value')
        
        self.graphics_checkbox = Checkbox(
            value=display_graphics,
            description='Show Graphics',
            disabled=False
        )
        self.graphics_checkbox.observe(self.toggle_graphics, names='value')
        
        self.status = HTML(value="<b>Status:</b> Ready")
        
        self.controls = VBox([
            HBox([self.start_button, self.step_button, self.reset_button, self.batch_button]),
            HBox([self.speed_slider, self.run_slider, self.graphics_checkbox]),
            self.status
        ])
        
        with self.output:
            frame, _ = self.game.step()
            if self.display_graphics:
                clear_output(wait=True)
                plt.figure(figsize=(12, 9))
                plt.imshow(frame)
                plt.axis('off')
                plt.title(f"Run: {self.game.run_id}/{self.game.max_runs} - Frame: {self.game.frame}", fontsize=14)
                plt.show()
            else:
                clear_output(wait=True)
                print(f"Run: {self.game.run_id}/{self.game.max_runs} - Frame: {self.game.frame}")
                print(f"Player in Room: {self.game.current_room}")
                print(f"Health: {int(self.game.player['health'])}")
                print(f"Resources: {self.game.player['resources_collected']}/4")
                print(f"Enemies: {self.game.player['enemies_killed']}/5")
                print(f"Door: {'Open' if self.game.door['open'] else 'Closed'}")
                print(f"NPC Emotion: {self.game.npc['emotion'].value}")
    
    def toggle_simulation(self, b):
        self.running = not self.running
        if self.running:
            self.status.value = "<b>Status:</b> Running"
            self.run_simulation()
        else:
            self.status.value = "<b>Status:</b> Paused"
    
    def run_simulation(self):
        if self.running:
            frame, done = self.game.step()
            with self.output:
                if self.display_graphics:
                    clear_output(wait=True)
                    plt.figure(figsize=(12, 9))
                    plt.imshow(frame)
                    plt.axis('off')
                    plt.title(f"Run: {self.game.run_id}/{self.game.max_runs} - Frame: {self.game.frame}", fontsize=14)
                    plt.show()
                else:
                    if self.game.frame % 10 == 0:
                        clear_output(wait=True)
                        print(f"Run: {self.game.run_id}/{self.game.max_runs} - Frame: {self.game.frame}")
                        print(f"Player in Room: {self.game.current_room}")
                        print(f"Health: {int(self.game.player['health'])}")
                        print(f"Resources: {self.game.player['resources_collected']}/4")
                        print(f"Enemies: {self.game.player['enemies_killed']}/5")
                        print(f"Door: {'Open' if self.game.door['open'] else 'Closed'}")
                        print(f"NPC Emotion: {self.game.npc['emotion'].value}")
            if done:
                self.running = False
                self.status.value = "<b>Status:</b> Complete"
            else:
                import IPython.display
                IPython.display.display(IPython.display.Javascript('setTimeout(function() { IPython.notebook.kernel.execute("simulation_interface.run_simulation()") }, 10)'))
    
    def step_simulation(self, b):
        with self.output:
            frame, done = self.game.step()
            if self.display_graphics:
                clear_output(wait=True)
                plt.figure(figsize=(12, 9))
                plt.imshow(frame)
                plt.axis('off')
                plt.title(f"Run: {self.game.run_id}/{self.game.max_runs} - Frame: {self.game.frame}", fontsize=14)
                plt.show()
            else:
                clear_output(wait=True)
                print(f"Run: {self.game.run_id}/{self.game.max_runs} - Frame: {self.game.frame}")
                print(f"Player in Room: {self.game.current_room}")
                print(f"Health: {int(self.game.player['health'])}")
                print(f"Resources: {self.game.player['resources_collected']}/4")
                print(f"Enemies: {self.game.player['enemies_killed']}/5")
                print(f"Door: {'Open' if self.game.door['open'] else 'Closed'}")
                print(f"NPC Emotion: {self.game.npc['emotion'].value}")
        if done:
            self.status.value = "<b>Status:</b> Complete"
        else:
            self.status.value = f"<b>Status:</b> Step {self.game.frame}"
    
    def run_batch(self, b):
        with self.output:
            frame, done = self.game.run_batch(100)
            if self.display_graphics:
                clear_output(wait=True)
                plt.figure(figsize=(12, 9))
                plt.imshow(frame)
                plt.axis('off')
                plt.title(f"Run: {self.game.run_id}/{self.game.max_runs} - Frame: {self.game.frame}", fontsize=14)
                plt.show()
            else:
                clear_output(wait=True)
                print(f"Run: {self.game.run_id}/{self.game.max_runs} - Frame: {self.game.frame}")
                print(f"Player in Room: {self.game.current_room}")
                print(f"Health: {int(self.game.player['health'])}")
                print(f"Resources: {self.game.player['resources_collected']}/4")
                print(f"Enemies: {self.game.player['enemies_killed']}/5")
                print(f"Door: {'Open' if self.game.door['open'] else 'Closed'}")
                print(f"NPC Emotion: {self.game.npc['emotion'].value}")
        if done:
            self.status.value = "<b>Status:</b> Complete"
        else:
            self.status.value = f"<b>Status:</b> Frame {self.game.frame}"
    
    def reset_simulation(self, b):
        self.running = False
        self.game.reset()
        with self.output:
            frame, _ = self.game.step()
            if self.display_graphics:
                clear_output(wait=True)
                plt.figure(figsize=(12, 9))
                plt.imshow(frame)
                plt.axis('off')
                plt.title(f"Run: {self.game.run_id}/{self.game.max_runs} - Frame: {self.game.frame}", fontsize=14)
                plt.show()
            else:
                clear_output(wait=True)
                print(f"Run: {self.game.run_id}/{self.game.max_runs} - Frame: {self.game.frame}")
                print(f"Player in Room: {self.game.current_room}")
                print(f"Health: {int(self.game.player['health'])}")
                print(f"Resources: {self.game.player['resources_collected']}/4")
                print(f"Enemies: {self.game.player['enemies_killed']}/5")
                print(f"Door: {'Open' if self.game.door['open'] else 'Closed'}")
                print(f"NPC Emotion: {self.game.npc['emotion'].value}")
        self.status.value = "<b>Status:</b> Reset"
    
    def update_speed(self, change):
        self.speed = change.new
        self.game.speed_multiplier = self.speed
        self.status.value = f"<b>Status:</b> Speed set to {self.speed}x"
    
    def update_max_runs(self, change):
        self.game.max_runs = change.new
        self.status.value = f"<b>Status:</b> Max runs set to {change.new}"
    
    def toggle_graphics(self, change):
        self.display_graphics = change.new
        self.status.value = f"<b>Status:</b> Graphics {'enabled' if self.display_graphics else 'disabled'}"
        self.step_simulation(None)
    
    def display(self):
        display(HTML(value="<h1>NPC Emotion Simulation</h1>"))
        display(HTML(value="<p>Use the buttons below to control the simulation. The simulation shows a player (blue) and NPC (green) navigating through rooms to collect resources (yellow) and defeat enemies (red).</p>"))
        display(self.controls, self.output)

In [26]:
## Create and run the simulation interface only if not autonomous
if not AUTONOMOUS_MODE:
    simulation_interface = SimulationInterface(max_runs=NUM_RUNS, speed=SPEED_MULTIPLIER, display_graphics=DISPLAY_GRAPHICS)
    simulation_interface.display()


In [27]:
## Batch processing function for large number of runs
def run_large_batch(num_runs=NUM_RUNS, frames_per_run=10800, display_progress=True):
    print(f"Starting batch processing for {num_runs} runs...")
    try:
        os.makedirs(DATA_DIRECTORY, exist_ok=True)
        output_dir = DATA_DIRECTORY
    except:
        output_dir = tempfile.gettempdir()
        print(f"Could not access {DATA_DIRECTORY}, using {output_dir} instead")
    
    start_time = time.time()
    
    for run in range(1, num_runs + 1):
        game = Game(run_id=run, max_runs=num_runs)
        frame_count = 0
        while not game.game_over and frame_count < frames_per_run:
            game.update()
            frame_count += 1
            if display_progress and (run % 10 == 0 and frame_count % 1000 == 0):
                elapsed = time.time() - start_time
                progress = (run - 1 + frame_count/frames_per_run) / num_runs
                estimated_total = elapsed / progress if progress > 0 else 0
                remaining = estimated_total - elapsed
                clear_output(wait=True)
                print(f"Run {run}/{num_runs} ({run/num_runs*100:.1f}%)")
                print(f"Frame {frame_count}/{frames_per_run} ({frame_count/frames_per_run*100:.1f}%)")
                print(f"Elapsed time: {elapsed/60:.1f} minutes")
                print(f"Estimated remaining: {remaining/60:.1f} minutes")
                print(f"Progress: {progress*100:.2f}%")
                bar_length = 50
                filled_length = int(bar_length * progress)
                bar = '█' * filled_length + '░' * (bar_length - filled_length)
                print(f"[{bar}]")
    
    total_time = time.time() - start_time
    print(f"Batch processing complete!")
    print(f"Total time: {total_time/60:.2f} minutes")
    print(f"CSV files saved to '{output_dir}' directory")
    return total_time

In [28]:
## Autonomous Simulation Trigger
if AUTONOMOUS_MODE:
    run_large_batch(NUM_RUNS, frames_per_run=10800, display_progress=True)

Starting batch processing for 2500 runs...
Room 2 setup: 2 resources and 5 enemies
Enemy killed! Total: 1/5
Enemy killed! Total: 2/5
Enemy killed! Total: 4/5
Door opened! Player can now move to Room 2
Transitioning to Room 2
Enemy killed! Total: 6/5
Enemy killed! Total: 9/5
Enemy killed! Total: 10/5
Run 1 completed successfully!
Room 2 setup: 2 resources and 5 enemies
Enemy killed! Total: 1/5
Enemy killed! Total: 2/5
Door opened! Player can now move to Room 2
Transitioning to Room 2
Enemy killed! Total: 5/5
Enemy killed! Total: 6/5
Run 2 completed successfully!
Room 2 setup: 2 resources and 5 enemies
Enemy killed! Total: 1/5
Enemy killed! Total: 2/5
Enemy killed! Total: 4/5
Door opened! Player can now move to Room 2
Enemy killed! Total: 5/5
Transitioning to Room 2
Enemy killed! Total: 6/5
Enemy killed! Total: 7/5
Run 3 completed successfully!
Run 3 completed successfully!
Room 2 setup: 2 resources and 5 enemies
Enemy killed! Total: 1/5
Enemy killed! Total: 2/5
Enemy killed! Total: 4/5
